<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preparing-files" data-toc-modified-id="Preparing-files-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preparing files</a></span><ul class="toc-item"><li><span><a href="#Evolução-internações-por-doença" data-toc-modified-id="Evolução-internações-por-doença-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Evolução internações por doença</a></span></li><li><span><a href="#Mortalidade" data-toc-modified-id="Mortalidade-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Mortalidade</a></span></li><li><span><a href="#Distribuição-de-idade" data-toc-modified-id="Distribuição-de-idade-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Distribuição de idade</a></span></li><li><span><a href="#Custo-das-internações" data-toc-modified-id="Custo-das-internações-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Custo das internações</a></span></li></ul></li></ul></div>

In [1]:
from tools import *

pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

import seaborn as sns; sns.set()
import matplotlib.pyplot as plt

%matplotlib inline

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

# Preparing files

In [2]:
sex_map, cnes_map, uf_map, cid_map, et_map, \
financ_map, cbo_map, comp_map, mun_map, caratend_map = make_maps()

In [3]:
# download_all() # uncomment to download all files
# make_hemato_ds() # uncomment to fetch a new hemato ds

In [4]:
pop_df = pop_ibge()

In [5]:
hemato = open_hemato_df()
hemato.UF_ZI = hemato.UF_ZI.map(uf_map)
hemato['DESCR_CID'] = hemato.DIAG_PRINC.map(cid_map)
hemato[['UF_ZI', 'DESCR_CID']] = hemato[['UF_ZI', 'DESCR_CID']].astype('category')

Optimizing df...

Original df size: Total memory usage: 435.46 MB
Optimized df size: Total memory usage: 52.20 MB


## Evolução internações por doença

In [7]:
ufs = ['Brasil']
ufs = ufs + (list(hemato.UF_ZI.values.categories))
ufs

@interact
def plot_cid_year(CID = hemato.DESCR_CID.values.categories, 
                  UF = ufs):
    if UF == 'Brasil':
        table = pd.pivot_table(hemato, values='N_AIH', index='DESCR_CID', columns=['ANO_CMPT'], 
                               aggfunc=np.count_nonzero)
    else:
        table = pd.pivot_table(hemato[hemato.UF_ZI == UF], 
                               values='N_AIH', index='DESCR_CID', columns=['ANO_CMPT'], 
                               aggfunc=np.count_nonzero)

    try:
        data = table.loc[CID]
    except KeyError:
        return print(f'Não existe registro de {CID} neste estado.')

    plt.figure(figsize=(10, 5))

    ax = sns.lineplot(data=data)
    plt.xlabel('ano')
    plt.ylabel(CID)
    plt.title(f"Internações por ano, {UF}:")
    
    return plt.show()

interactive(children=(Dropdown(description='CID', options=('Anemia refratária com excesso de blastos', 'Anemia…

## Mortalidade

In [9]:
ufs = ['Brasil']
ufs = ufs + (list(hemato.UF_ZI.values.categories))
ufs

@interact
def plot_cid_year(CID = hemato.DESCR_CID.values.categories, 
                  UF = ufs):
    if UF == 'Brasil':
        table = pd.pivot_table(hemato, values='MORTE', index='DESCR_CID', 
                               columns=['ANO_CMPT'], aggfunc=np.mean)
    else:
        table = pd.pivot_table(hemato[hemato.UF_ZI == UF], 
                               values='MORTE', index='DESCR_CID', 
                               columns=['ANO_CMPT'], aggfunc=np.mean)
    try:
        data = table.loc[CID]
    except KeyError:
        return print(f'Não existe registro de {CID} neste estado.')

    plt.figure(figsize=(10, 5))

    ax = sns.lineplot(data=data)
    plt.xlabel('ano')
    plt.ylabel(CID)
    plt.title(f"Índice de mortalidade por ano, {UF}:")
    
    return plt.show()

interactive(children=(Dropdown(description='CID', options=('Anemia refratária com excesso de blastos', 'Anemia…

## Distribuição de idade

In [10]:
ufs = ['Brasil']
ufs = ufs + (list(hemato.UF_ZI.values.categories))

years = ['Todos']
years = years + (list(hemato.ANO_CMPT.values.categories))

@interact
def plot_cid_year(CID = hemato.DESCR_CID.values.categories, 
                  UF = ufs, ANO = years):
    plt.figure(figsize=(10, 5))
    try:
        if UF == 'Brasil' and ANO == 'Todos':
            ax = sns.distplot(hemato[hemato.DESCR_CID == CID].IDADE, kde=False)
        elif UF == 'Brasil' and ANO != 'Todos':
            ax = sns.distplot(hemato[(hemato.DESCR_CID == CID) & (hemato.ANO_CMPT == ANO)].IDADE, 
                              kde=False)
        elif UF != 'Brasil' and ANO == 'Todos':
            sns.distplot(hemato[(hemato.DESCR_CID == CID) & (hemato.UF_ZI == UF)].IDADE, kde=False)
        else:
            sns.distplot(hemato[(hemato.DESCR_CID == CID) & (hemato.UF_ZI == UF) & (hemato.ANO_CMPT == ANO)].IDADE, 
                         kde=False)
    except KeyError:
        return print(f'Não existe registro de {CID}, {UF}.')
    plt.xlabel('Idade')
    plt.ylabel('Número de internações')
    plt.title(f"Distribuição por idade, {CID}, {UF}:")
    
    return plt.show()

interactive(children=(Dropdown(description='CID', options=('Anemia refratária com excesso de blastos', 'Anemia…

## Custo das internações

In [11]:
ufs = ['Brasil']
ufs = ufs + (list(hemato.UF_ZI.values.categories))
ufs

@interact
def plot_cid_year(CID = hemato.DESCR_CID.values.categories, 
                  UF = ufs, ANO = years):
    plt.figure(figsize=(10, 5))
    try:
        if UF == 'Brasil' and ANO == 'Todos':
            ax = sns.distplot(hemato[hemato.DESCR_CID == CID].US_TOT, kde=False)
        elif UF == 'Brasil' and ANO != 'Todos':
            ax = sns.distplot(hemato[(hemato.DESCR_CID == CID) & (hemato.ANO_CMPT == ANO)].US_TOT, 
                              kde=False)
        elif UF != 'Brasil' and ANO == 'Todos':
            sns.distplot(hemato[(hemato.DESCR_CID == CID) & (hemato.UF_ZI == UF)].US_TOT, kde=False)
        else:
            sns.distplot(hemato[(hemato.DESCR_CID == CID) & (hemato.UF_ZI == UF) & (hemato.ANO_CMPT == ANO)].US_TOT, 
                         kde=False)
    except KeyError:
        return print(f'Não existe registro de {CID}, {UF}.')
    plt.ylabel('Número de internações, escala logarítmica')
    plt.yscale('log')
    plt.xlabel('Repasse de AIH, em US$')
    plt.title(f"Repasse de AIH, em dólar, {CID}, {UF}:")
    
    return plt.show()

interactive(children=(Dropdown(description='CID', options=('Anemia refratária com excesso de blastos', 'Anemia…